## Building Research Agent tool

In [2]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os
import sendgrid
import asyncio
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [3]:
load_dotenv(override=True)

True

### We'd be utilizing below tools from SDK:

1. WebSearchTool
2. FileSearchTool
3. ComputerTool

In [4]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [5]:
message = "Latest AI Agent frameworks in 2025"

with trace('search'):
    result = await Runner.run(search_agent, message)
    display(Markdown(f"**Search Agent Result:** {result}"))

**Search Agent Result:** RunResult:
- Last agent: Agent(name="Search agent", ...)
- Final output (str):
    As of August 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems:
    
    - **LangChain**: A modular framework for building applications powered by large language models (LLMs). It provides tools for chaining prompts, models, memory, and external tools, facilitating the creation of complex workflows. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))
    
    - **LangGraph**: An extension of LangChain, LangGraph focuses on stateful, graph-based agent systems. It offers explicit control over agent workflows, supporting branching and debugging of complex behaviors. ([radarmagazine.com](https://www.radarmagazine.com/top-5-ai-agent-frameworks-it-executives-should-be-watching-in-2025/?utm_source=openai))
    
    - **CrewAI**: This framework adopts a role-based agent collaboration approach, enabling the creation of specialized agents that work together on complex projects, similar to a team environment. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))
    
    - **AutoGen**: Developed by Microsoft, AutoGen specializes in orchestrating multiple AI agents to form autonomous, event-driven systems capable of handling complex, multi-agent tasks seamlessly. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))
    
    - **Eliza**: A Web3-friendly AI agent operating system that integrates seamlessly with blockchain applications, allowing for the deployment of decentralized AI agents. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))
    
    - **Agent Lightning**: A flexible and extensible framework that enables reinforcement learning-based training of LLMs for any AI agent, facilitating complex interaction logic and multi-agent scenarios. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))
    
    - **AutoAgent**: A fully-automated, zero-code framework for LLM agents, allowing users to create and deploy agents through natural language alone, making AI agent development accessible to non-technical users. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))
    
    These frameworks represent the forefront of AI agent development, each catering to different needs and applications in the rapidly evolving field of artificial intelligence. 
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)

In [6]:
number_searches = 3 #To avoid getting charged for too many searches

INSTRUCTIONS = f"You are a helpful research assistant.  Given a query, come up with a set of web searches \
to perform to best answer the query. Output {number_searches} terms to query for."

class WebSearchItem(BaseModel):
    reason: str = Field(description="Why this search term is relevant to the query")
    query: str = Field(description="The search term to use")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description=f"A list of {number_searches} search terms to query for")


plan_agent = Agent(
    name="Planneragent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

# planner_agent = Agent(
#     name="PlannerAgent",
#     instructions=INSTRUCTIONS,
#     model="gpt-4o-mini",
#     output_type=WebSearchPlan,
# )

In [ ]:
message = "Latest AI Agent frameworks in 2025"

with trace('search'):
    result = await Runner.run(plan_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find a comprehensive list of AI agent frameworks released or updated in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore expert analyses and reviews of AI agents that are leading in 2025.', query='best AI agents frameworks reviews 2025'), WebSearchItem(reason='To gather information about emerging trends and technologies in AI agent development for 2025.', query='trends in AI agent frameworks 2025')]


In [25]:
@function_tool
def send_email1(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = os.getenv("YAHOO_ID")
    to_email = os.getenv("GMAIL_ID")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}


In [28]:
send_email1

FunctionTool(name='send_email1', description='Send out an email with the given subject and HTML body to all sales prospects', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1110460c0>, strict_json_schema=True, is_enabled=True)

In [26]:
# Creating email agent that sends nicely formatted HTML email...

INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email1],
    model="gpt-4o-mini",
)



In [11]:
# Creating report writer agent...
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportOutline(BaseModel):
    short_summary: str = Field(description="A brief summary of the report, 2-3 sentences")
    markdown_report_outline: str = Field(description="A detailed outline of the report, in markdown format")
    follow_up_questions: list[str] = Field(description="A list of 3-5 follow up questions that could be explored in future research")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportOutline
)

In [20]:
# Below 3 functions will plan and execute the search using planner and search agents

async def plan_search(query: str):
    """This function uses the planner agent to plan which searches to perform"""
    print("Planning searches...")
    plan_result = await Runner.run(plan_agent, f"query: {query}")
    print(f"Planner agent will perform {len(plan_result.final_output.searches)} searches")
    return plan_result.final_output


async def execute_search(plan: WebSearchPlan):
    """This function will call search() for each item in the plan, and return the results"""
    print("searching...")
    tasks = [asyncio.create_task(Runner.run(search_agent, f"search term: {item.query}")) for item in plan.searches]
    results = await asyncio.gather(*tasks)
    print("searches complete")
    return results


async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output
    


In [33]:
# These 2 functions will create the report and email it

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportOutline):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report_outline)
    print("Email sent")
    return report

In [34]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_search(query)
    search_results = await execute_search(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")




Starting research...
Planning searches...
Planner agent will perform 3 searches
searching...
searches complete
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!
